In [10]:
root='/home/oliverhu/wangfeng/SSTG_re/data_save/data_save/deap/20230411_09:33:27'


import numpy as np
import os
def get_results(log_path):
    if os.path.exists(log_path):
        acc_array=[]
        with open(log_path) as f:
            for line in f:
                if ' acc= ' in line:
                    acc=float(line.split('acc= ')[1].split(' ')[0])
                    acc_array.append(acc)

        acc=np.mean(acc_array)
        std=np.std(acc_array)*100
        os.rename(log_path,os.path.join(root,f'{acc:.4f}_{std:.4f}.log'))
get_results(os.path.join(root,'log.log'))

In [ ]:
# draw confusion matrix
import matplotlib.pyplot as pl
import sklearn.metrics as metrics
import torch
import os

import numpy as np
def plot_matrix(file_name,is_train,y_true, y_pred, labels_name, cm_root,title=None, thresh=0.8, axis_labels=None,):
    # 利用sklearn中的函数生成混淆矩阵并归一化

    cm = metrics.confusion_matrix(
        y_true, y_pred, labels=labels_name, sample_weight=None)  # 生成混淆矩阵
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]  # 归一化
    print(cm)
    # 画图，如果希望改变颜色风格，可以改变此部分的cmap=pl.get_cmap('Blues')处
    pl.imshow(cm, interpolation='nearest', cmap=pl.get_cmap('Blues'))
    pl.colorbar()  # 绘制图例
    # pl.figure(figsize=(10, 10))
    # 图像标题
    if title is not None:
        pl.title(title)
    # 绘制坐标
    num_local = np.array(range(len(labels_name)))
    # if axis_labels is None:
    #     axis_labels = labels_name
    if len(labels_name) == 2:
        axis_labels = ['negative', 'positive']
    elif len(labels_name) == 3:
        axis_labels = ['negative', 'neutral', 'positive']
    pl.xticks(num_local, axis_labels)  # 将标签印在x轴坐标上， 并倾斜45度
    pl.yticks(num_local, axis_labels)  # 将标签印在y轴坐标上
    pl.ylabel('True label')
    pl.xlabel('Predicted label')

    # 将百分比打印在相应的格子内，大于thresh的用白字，小于的用黑字
    for i in range(np.shape(cm)[0]):
        for j in range(np.shape(cm)[1]):
            if int(cm[i][j] * 100) > 0:
                pl.text(j, i, format(round(cm[i][j] * 100, 2), '.2f') + '%',
                        ha="center", va="center",
                        color="white" if cm[i][j] > thresh else "black")  # 如果要更改颜色风格，需要同时更改此行
    # 显示
    # pl.show()
    pl.savefig(cm_root+'/{}_{}.pdf'.format(file_name,is_train))
    pl.close()

cm_root=os.path.join(root,'cm')
dirlist=os.listdir(cm_root)

if 'cm.pt' not in dirlist:
    pre_train_all=[]
    true_train_all=[]
    pre_test_all=[]
    true_test_all=[]
    for dir in dirlist:
        data=torch.load(os.path.join(cm_root,dir))
        pre_train_all.append(data[0])
        true_train_all.append(data[1])
        pre_test_all.append(data[2])
        true_test_all.append(data[3])

    pre_train_all=np.concatenate(pre_train_all,0)
    true_train_all=np.concatenate(true_train_all,0)
    pre_test_all=np.concatenate(pre_test_all,0)
    true_test_all=np.concatenate(true_test_all,0)

    torch.save([pre_train_all,true_train_all,pre_test_all,true_test_all],os.path.join(cm_root,'cm.pt'))


data=torch.load(os.path.join(cm_root,'cm.pt'))
pre_train,true_train,pre_test,true_test=data[0],data[1],data[2],data[3]
print(pre_test.shape,true_test.shape)

#plot train
plot_matrix('cm','train',true_train,pre_train,[i for i in range(2)],cm_root)
#plot test
plot_matrix('cm','test',true_test,pre_test,[i for i in range(2)],cm_root)


In [8]:

labels_name=[i for i in range(3)]
cm_root=os.path.join(root,'cm')
data=torch.load(os.path.join(cm_root,'cm.pt'))
pre_train,true_train,pre_test,true_test=data[0],data[1],data[2],data[3]
cm = metrics.confusion_matrix(
        pre_test, true_test, labels=labels_name, sample_weight=None)  # 生成混淆矩阵
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]  # 归一化
pl.imshow(cm, interpolation='nearest', cmap=pl.get_cmap('Blues'))
pl.colorbar()  # 绘制图例

# 绘制坐标


num_local = np.array(range(len(labels_name)))
# if axis_labels is None:
#     axis_labels = labels_name
if len(labels_name) == 2:
    axis_labels = ['negative', 'positive']
elif len(labels_name) == 3:
    axis_labels = ['negative', 'neutral', 'positive']
pl.xticks(num_local, axis_labels)  # 将标签印在x轴坐标上， 并倾斜45度
pl.yticks(num_local, axis_labels)  # 将标签印在y轴坐标上
pl.ylabel('True label')
pl.xlabel('Predicted label')

# 将百分比打印在相应的格子内，大于thresh的用白字，小于的用黑字
for i in range(np.shape(cm)[0]):
    for j in range(np.shape(cm)[1]):
        if int(cm[i][j]*100) > 0:
            pl.text(j, i, f'{cm[i][j]*100:.2f}' + '%',
                    ha="center", va="center",
                    color="white" if cm[i][j] > 0.8 else "black")  # 如果要更改颜色风格，需要同时更改此行
# 显示
# pl.show()
pl.savefig('/home/oliverhu/wangfeng/SSTG_re/visualize/seed_cm_indpendent.pdf')
pl.close()          


In [ ]:
import torch 
data=torch.load('/home/oliverhu/wangfeng/SSTG_re/data_save/data_save/seed/20230421_16:48:28/cm/cm.pt')
pre_train,true_train,pre_test,true_test=data[0],data[1],data[2],data[3]
print(pre_test.shape,true_test.shape)
print(type(pre_test))
#plot test
plot_matrix('cm','test',true_test,pre_test,[i for i in range(2)],cm_root)